In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import datetime

In [2]:
import fates_xarray_funcs
from fates_xarray_funcs import monthly_to_annual

In [3]:
cases_in = {'S0':'f19_0007_trendyS0_39e91e09b5_c2da27fd',
         'S1':'f19_0008_trendyS1_39e91e09b5_c2da27fd',
         'S2':'f19_0010_trendyS2_hybrid_39e91e09b5_c2da27fd',
         'S3':'f19_0009_trendyS3_parta_39e91e09b5_c2da27fd',
         'S3b':'f19_0011_trendyS3_partb_hybrid_39e91e09b5_c2da27fd'}

cases_out = ['S0','S1','S2','S3']

start_year = {'S0':1701,
         'S1':1701,
         'S2':1901,
         'S3':1701,
         'S3b':1901}

base_data_dir = '/global/homes/c/cdkoven/cdkoven_m2467/trendy_2025_tseries_files/'

minlat = -60.

In [4]:
output_vars_list = ['tas', 'pr', 'rsds', 'mrso', 'mrro', 'evapotrans', 'evapo', 'cVeg', 'cLitter', 'cSoil', 'cProduct', 
                    'gpp', 'ra', 'npp', 'rh', 'fFire', 'fLuc', 'soilr', 'nbp', 
                    'landCoverFrac', 'oceanCoverFrac', 'burntArea', 'lai', 
                    'cLeaf', 'cWood', 'cRoot', 'cCwd', 'cSoilpools', 'fVegLitter', 'fLeafLitter', 'fWoodLitter', 
                    'fRootLitter', 'fLitterSoil', 'fVegSoil', 'rhpool', 'fAllocLeaf', 'fAllocWood', 'fAllocRoot', 
                    'fFireCveg', 'fFireLitter', 'fFireCsoil', 'tsl', 'msl', 'evspsblveg', 'evspsblsoi', 'tran', 
                    'fGrazing']

output_pft_vars_list = ['evapotranspft', 'transpft', 'albedopft', 'snow_depthpft', 'shflxpft', 'rnpft',  'cVegpft', 'cSoilpft',
                        'gpppft', 'npppft', 'rhpft', 'nbppft', 'laipft',   'tskinpft', 'mslpft', 'theightpft']

input_vars_list = ['FATES_GPP','TSA','RAIN','SNOW','NBP','area','FATES_FRACTION','FATES_NPP','FATES_AUTORESP','HR','FATES_SEEDS_IN_EXTERN_EL']



In [5]:
unitslist = ['K','kg m-2 s-1','W m-2','kg m-2','fraction','m m-2','m']

units_conversions = {'gC/m^2/s':['kg m-2 s-1', 1e-3],
                     'mm/s':['kg m-2 s-1', 9999],
}


In [6]:
def gettime():
    return datetime.datetime.now().strftime("%Y-%m-%d_%H.%M.%S.%f")

def inherit_attrs(parents):
    attrs = parents[0].attrs
    for parent in parents[1:]:
        attrs.update({key: value for (key, value) in parent.attrs.items() if 'history_' in key})
    return attrs
        
class datadict(dict):
    def getdata(self, varname, cases_in=cases_in, cases_out=cases_out, start_year=start_year, base_data_dir=base_data_dir):
        fins = {}
        dataarrays_out = {}
        for case_in in cases_in:
            fins[case_in] = xr.open_dataset(base_data_dir+cases_in[case_in]+'/'+cases_in[case_in]+'.tseries.'+varname+'.nc')
        for case_out in cases_out:
            if case_out in ['S0','S1'] or 'time' not in fins[case_out][varname].dims:
                dataset_out = fins[case_out]
            elif case_out == 'S2':
                s2a = fins['S1'].isel(time=slice(0,200*12))
                s2b = fins['S2']
                dataset_out = xr.concat((s2a,s2b), dim='time')
            elif case_out == 'S3':
                s3a = fins['S3'].isel(time=slice(0,200*12))
                s3b = fins['S3b']
                dataset_out = xr.concat((s3a,s3b), dim='time')
            dataset_out['time'] = np.arange(len(dataset_out['time']))/12.+1701.
            dataset_out = dataset_out.sel(lat=slice(minlat,90.))
            if "fates_levelem" in dataset_out.dims:
                dataset_out = dataset_out.isel(fates_levelem=0).squeeze()
            dataarrays_out.update({case_out:dataset_out[varname]})
        self.update({varname:dataarrays_out})

    def fates_frac_scale(self, varname):
        dataarrays_out = {}
        if varname[0:6] == 'FATES_':
            newvarname = varname[6:]
        else:
            raise Exception
        for case in self[varname]:
            scaledvar = self[varname][case] * self['FATES_FRACTION'][case]
            scaledvar.attrs = inherit_attrs((self[varname][case],))
            scaledvar = scaledvar.rename(newvarname)
            scaledvar.attrs['history_'+gettime()] = newvarname + ' = ' + varname + ' * FATES_FRACTION'
            dataarrays_out[case] = scaledvar
        self.update({newvarname:dataarrays_out})

    def writedata(self, varname, output_data_dir=base_data_dir+'/postprocessed/'):
        for case_out in self[varname]:
            #try:
            self[varname][case_out].to_netcdf(output_data_dir+'ELM-FATES_'+case_out+'_'+varname+'.nc', format="NETCDF4_CLASSIC", engine='netcdf4', encoding={varname:{"zlib": True,"complevel":1}})
            #except:
                #self[varname][case_out].to_netcdf(output_data_dir+'ELM-FATES_'+case_out+'_'+varname+'.nc', format="NETCDF4_CLASSIC", engine='netcdf4')

    def rename(self, oldvarname, newvarname):
        self[newvarname] = {}
        for case in self[oldvarname]:
            self[newvarname][case] = self[oldvarname][case].rename(newvarname)
            self[newvarname][case].attrs['history_'+gettime()] = newvarname + ' = ' + oldvarname

    def addvars(self, var1name, var2name, newvarname):
        self[newvarname] = {}
        for case in self[var1name]:
            if self[var1name][case].attrs['units'] != self[var2name][case].attrs['units']:
                print("units need to match for vars "+var1name + " and "+var2name)
                print(self[var1name][case].attrs['units'])
                print(self[var2name][case].attrs['units'])
                raise Exception
            newvar = self[var1name][case] + self[var2name][case]
            newvar.attrs = inherit_attrs((self[var1name][case], self[var2name][case]))
            newvar = newvar.rename(newvarname)
            newvar.attrs['long_name'] = newvarname
            newvar.attrs['history_'+gettime()] = newvarname + ' = ' + var1name + ' + ' + var2name
            self[newvarname][case]  = newvar

    def check_units(self):
        for var in self:
            for case in self[var]:
                if self[var][case].attrs['units'] not in unitslist:
                    print(var,case,self[var][case].attrs['units'])

    def fix_units(self, varname):
        for case in self[varname]:
            if self[varname][case].attrs['units'] in unitslist:
                print('no need to convert units for variable '+varname + 'with units: ' + self[varname][case].attrs['units'])
            else:
                if self[varname][case].attrs['units'] in units_conversions:
                    newunits, scale = units_conversions[self[varname][case].attrs['units']]
                    oldunits = self[varname][case].attrs['units']
                    self[varname][case] = self[varname][case] * scale
                    self[varname][case].attrs['units'] = newunits
                    self[varname][case].attrs['history_'+gettime()] = 'unit conversion on '+varname+' of '+str(scale)+' from: '+ oldunits + ' to: ' + newunits
                else:
                    print('units not in conversion list: '+self[varname][case].attrs['units'])
                    raise Exception

    


In [7]:
varblock = datadict()

In [8]:
for var in input_vars_list:
    varblock.getdata(var)


In [9]:
varblock.check_units()

RAIN S0 mm/s
RAIN S1 mm/s
RAIN S2 mm/s
RAIN S3 mm/s
SNOW S0 mm/s
SNOW S1 mm/s
SNOW S2 mm/s
SNOW S3 mm/s
NBP S0 gC/m^2/s
NBP S1 gC/m^2/s
NBP S2 gC/m^2/s
NBP S3 gC/m^2/s
area S0 km^2
area S1 km^2
area S2 km^2
area S3 km^2
FATES_FRACTION S0 m2 m-2
FATES_FRACTION S1 m2 m-2
FATES_FRACTION S2 m2 m-2
FATES_FRACTION S3 m2 m-2
HR S0 gC/m^2/s
HR S1 gC/m^2/s
HR S2 gC/m^2/s
HR S3 gC/m^2/s


In [10]:
varblock.fates_frac_scale('FATES_SEEDS_IN_EXTERN_EL')
varblock.fix_units('NBP')
varblock.addvars('NBP','SEEDS_IN_EXTERN_EL','nbp')
varblock.writedata('nbp')

In [11]:
varblock.addvars('RAIN','SNOW','pr')
varblock.writedata('pr')

In [12]:
varblock.fates_frac_scale('FATES_GPP')
varblock.addvars('GPP','SEEDS_IN_EXTERN_EL','gpp')
varblock.writedata('gpp')

In [13]:
varblock.rename('TSA', 'tas')
varblock.writedata('tas')

In [14]:
varblock.fates_frac_scale('FATES_NPP')
varblock.addvars('NPP','SEEDS_IN_EXTERN_EL','npp')
varblock.writedata('npp')